## IPL Win Predictor

### Data Parsing and Cleaning

In [238]:
# importing the libraries
import numpy as np
import pandas as pd

In [239]:
# importing the dataset
match = pd.read_csv('matches.csv')
delivery = pd.read_csv('deliveries.csv')

In [240]:
match.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [241]:
match.shape

(756, 18)

In [242]:
delivery.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [243]:
#calculating total runs inning wise in each match by using group by
total_score_df = delivery.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()

C:\Users\Saket\AppData\Local\Temp\ipykernel_3852\3980492545.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_score_df = delivery.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()


In [244]:
# removing scores of the seconfd innings as not required
total_score_df = total_score_df[total_score_df['inning'] == 1]

In [245]:
# merging total_score_df in matches dataset
match_df = match.merge(total_score_df[['match_id', 'total_runs']],left_on="id", right_on="match_id")

#### Removing teams which are not playing and chainging names of team whose names are changed

In [246]:
match_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [247]:
match_df.shape

(756, 20)

In [248]:
# making list of currently playing teams
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders'
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [249]:
# replacing team names for delhi and heydrabad
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils', 'Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')



In [250]:
# removing all other teams which are not playing
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [251]:
match_df.shape

(342, 20)

In [252]:
# removing matches that were affected by rains
match_df = match_df[match_df['dl_applied'] == 0]

In [253]:
# extracting cols which are needed from match_df
match_df = match_df[['match_id', 'city', 'winner', 'total_runs' ]]

In [254]:
# assigning match-dfwith delivery
delivery_df = match_df.merge(delivery, on='match_id')

In [255]:
#for chasing innings data
delivery_df = delivery_df[delivery_df['inning'] == 2]

In [256]:
#Cummilative Sum function
delivery_df['current_score'] = delivery_df.groupby('match_id')['total_runs_y'].cumsum()

In [257]:
#Runs Left
delivery_df['runs_left'] = delivery_df['total_runs_x'] - delivery_df['current_score']

In [258]:
#Balls Left
delivery_df['balls_left'] = 126 - (delivery_df['over']*6 + delivery_df['ball'])

In [259]:
#Wickets Left
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')
wickets = delivery_df.groupby('match_id')['player_dismissed'].cumsum().values
delivery_df['wickets'] = 10 - wickets
delivery_df.head()

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,1,0,1,0,NaN,NaN,1,206,119,10
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,206,118,10
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,206,117,10
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,2,0,2,0,NaN,NaN,3,204,116,10
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,4,0,4,0,NaN,NaN,7,200,115,10


In [260]:
#Current Run Rate
delivery_df['crr'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls_left'])

In [261]:
#Required Run Rate
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

In [262]:
#Result
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [263]:
delivery_df['result'] = delivery_df.apply(result,axis=1)

In [264]:
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [265]:
final_df.sample(final_df.shape[0])

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result
71796,Chennai Super Kings,Delhi Daredevils,Delhi,80,42,8,171,7.000000,11.428571,0
72020,Royal Challengers Bangalore,Rajasthan Royals,Jaipur,96,65,7,174,8.509091,8.861538,0
32853,Mumbai Indians,Deccan Chargers,Visakhapatnam,68,41,7,138,5.316456,9.951220,1
69046,Delhi Daredevils,Chennai Super Kings,Pune,196,99,9,218,6.285714,11.878788,0
73190,Mumbai Indians,Delhi Capitals,Mumbai,172,86,8,219,8.294118,12.000000,0
...,...,...,...,...,...,...,...,...,...,...
60268,Delhi Daredevils,Sunrisers Hyderabad,Raipur,90,48,6,163,6.083333,11.250000,0
15455,Deccan Chargers,Delhi Daredevils,Durban,89,59,8,173,8.262295,9.050847,0
52151,Chennai Super Kings,Rajasthan Royals,Ranchi,61,48,7,148,7.250000,7.625000,1
41268,Royal Challengers Bangalore,Delhi Daredevils,Bangalore,103,83,8,152,7.945946,7.445783,1


In [266]:
final_df.dropna(inplace=True)

C:\Users\Saket\AppData\Local\Temp\ipykernel_3852\2709626079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(inplace=True)


In [267]:
final_df= [final_df['balls_left']!=0]

In [269]:
X = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=1)

AttributeError: 'list' object has no attribute 'iloc'

In [ ]:
X_train

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr
12375,Rajasthan Royals,Delhi Daredevils,Centurion,30,23,5,143,6.989691,7.826087
59979,Mumbai Indians,Chennai Super Kings,Chennai,127,98,10,158,8.454545,7.775510
53338,Sunrisers Hyderabad,Royal Challengers Bangalore,Hyderabad,126,99,10,160,9.714286,7.636364
46323,Rajasthan Royals,Sunrisers Hyderabad,Hyderabad,84,47,7,136,4.273973,10.723404
54744,Delhi Daredevils,Chennai Super Kings,Chennai,135,106,10,150,6.428571,7.641509
...,...,...,...,...,...,...,...,...,...
16221,Chennai Super Kings,Mumbai Indians,Port Elizabeth,41,29,7,147,6.989011,8.482759
67408,Royal Challengers Bangalore,Delhi Daredevils,Bengaluru,67,40,7,186,8.925000,10.050000
10877,Royal Challengers Bangalore,Chennai Super Kings,Port Elizabeth,123,61,5,179,5.694915,12.098361
25339,Royal Challengers Bangalore,Mumbai Indians,Mumbai,169,111,10,184,10.000000,9.135135


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf= ColumnTransformer([
   ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
pipe= Pipeline(steps=[
    ('step1',trf),
    ('step2', LogisticRegression(solver='liblinear'))
])

In [ ]:
pipe.fit(X_train,y_train)

C:\Users\Saket\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Saket\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values